In [ ]:
!pip3 install datasets

In [36]:
import glob
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [37]:
csv_files = glob.glob("AttackRangeCSV/malicious/*.csv")

malicious_df = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    malicious_df = pd.concat([malicious_df, df])

malicious_df.drop(['process_id', 'parent_process_id', 'count', 'firstTime', 'min(_time)', 'process_guid', 'parent_process_guid', 'lastTime'],  axis=1, inplace=True)

malicious_df['prompt'] = malicious_df[malicious_df.columns[0:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

malicious_df['completion'] = 1
malicious_df.drop(['parent_process', 'process_name', 'user', 'dest', 'process', 'parent_process_name', 'original_file_name'], axis=1, inplace=True)


In [38]:
csv_files = glob.glob("AttackRangeCSV/normal/*.csv")

normal_df = pd.DataFrame()

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    normal_df = pd.concat([normal_df, df])

normal_df.drop(['process_id', 'parent_process_id', 'count', 'firstTime', 'lastTime'],  axis=1, inplace=True)

normal_df['prompt'] = normal_df[normal_df.columns[0:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

normal_df['completion'] = 0
normal_df.drop(['parent_process', 'process_name', 'user', 'dest', 'process', 'parent_process_name', 'original_file_name'], axis=1, inplace=True)


In [42]:
openai_df = pd.concat([malicious_df[:60], normal_df[:60]])
openai_df = openai_df.sample(frac=1)
openai_df.to_json("SplunkAttackRangeAlerts/openaiDataset.jsonl", orient='records', lines=True)

In [48]:
hf_training_df, hf_testing_df = train_test_split(openai_df, test_size=0.2)

hf_training_df.rename(columns={'prompt': 'text', 'completion': 'label'}, inplace=True)
hf_training_df.to_json("SplunkAttackRangeAlerts/hfTrainingDataset.jsonl", orient='records', lines=True)

hf_testing_df.rename(columns={'prompt': 'text', 'completion': 'label'}, inplace=True)
hf_testing_df.to_json("SplunkAttackRangeAlerts/hfTestingDataset.jsonl", orient='records', lines=True)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dataset = load_dataset('json', data_files={'train': 'SplunkAttackRangeAlerts/hfTrainingDataset.jsonl', 'test': 'SplunkAttackRangeAlerts/hfTestingDataset.jsonl'})
dataset.push_to_hub('chathuru/SplunkAttackRangeAlerts')